#A notebook implementation of Keras-Retinanet Evalute.py

#Downgrade Tensorflow and Keras

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.3.0


In [ ]:
!pip uninstall keras-nightly

!pip uninstall -y tensorflow

!pip install h5py==2.10.0  


In [ ]:
!pip uninstall tensorflow
!pip uninstall keras

In [ ]:
!pip install tensorflow==2.3.0
!pip install keras==2.4.3

#Download keras-retinannet and dependencies

In [ ]:
!git clone https://github.com/SherilK/keras-retinanet.git

In [ ]:
%cd keras-retinanet/

!pip install keras_retinanet
!pip install keras-resnet

In [ ]:
!python setup.py build_ext --inplace

#EVALUATE

In [ ]:
import argparse
import os
import sys

# Allow relative imports when being executed as script.
'''
if __name__ == "__main__" and __package__ is None:
    sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..', '..'))
    import keras_retinanet.bin  # noqa: F401
    __package__ = "keras_retinanet.bin"
'''
# Change these to absolute imports if you copy this script outside the keras_retinanet package.
from keras_retinanet import models
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.utils.anchors import make_shapes_callback
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters, parse_pyramid_levels
#from keras_retinanet.utils.eval import evaluate
from keras_retinanet.utils.gpu import setup_gpu
from keras_retinanet.utils.tf_version import check_tf_version

In [ ]:
from keras_retinanet.utils.anchors import compute_overlap
from keras_retinanet.utils.visualization import draw_detections, draw_annotations

from tensorflow import keras
import numpy as np
import os
import time

import cv2
import progressbar
assert(callable(progressbar.progressbar)), "Using wrong progressbar module, install 'progressbar2' instead."

In [ ]:

from  keras_retinanet.utils.colors import label_color

#Visualization

In [ ]:

def draw_caption(image, box, caption):
    """ Draws a caption above the box in an image.

    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)


In [ ]:

def draw_box(image, box, color, thickness=2):
    """ Draws a box on an image with a given color.

    # Arguments
        image     : The image to draw on.
        box       : A list of 4 elements (x1, y1, x2, y2).
        color     : The color of the box.
        thickness : The thickness of the lines to draw a box with.
    """
    b = np.array(box).astype(int)
    cv2.rectangle(image, (b[0], b[1]), (b[2], b[3]), color, thickness, cv2.LINE_AA)
    

In [ ]:

def draw_detections(image, boxes, scores, labels, color=None, label_to_name=None, score_threshold=0.5):
    """ Draws detections in an image.

    # Arguments
        image           : The image to draw on.
        boxes           : A [N, 4] matrix (x1, y1, x2, y2).
        scores          : A list of N classification scores.
        labels          : A list of N labels.
        color           : The color of the boxes. By default the color from keras_retinanet.utils.colors.label_color will be used.
        label_to_name   : (optional) Functor for mapping a label to a name.
        score_threshold : Threshold used for determining what detections to draw.
    """
    selection = np.where(scores > score_threshold)[0]

    for i in selection:
        c = color if color is not None else label_color(labels[i])
        draw_box(image, boxes[i, :], color=c)

        # draw labels
        caption = (label_to_name(labels[i]) if label_to_name else labels[i]) + ': {0:.2f}'.format(scores[i])
        draw_caption(image, boxes[i, :], caption)


In [ ]:
'''
def draw_annotations(image, annotations, color=(0, 255, 0), label_to_name=None):
    """ Draws annotations in an image.

    # Arguments
        image         : The image to draw on.
        annotations   : A [N, 5] matrix (x1, y1, x2, y2, label) or dictionary containing bboxes (shaped [N, 4]) and labels (shaped [N]).
        color         : The color of the boxes. By default the color from keras_retinanet.utils.colors.label_color will be used.
        label_to_name : (optional) Functor for mapping a label to a name.
    """
    if isinstance(annotations, np.ndarray):
        annotations = {'bboxes': annotations[:, :4], 'labels': annotations[:, 4]}

    assert('bboxes' in annotations)
    assert('labels' in annotations)
    assert(annotations['bboxes'].shape[0] == annotations['labels'].shape[0])

    for i in range(annotations['bboxes'].shape[0]):
        label   = annotations['labels'][i]
        c       = color if color is not None else label_color(label)
        caption = '{}'.format(label_to_name(label) if label_to_name else label)
        draw_caption(image, annotations['bboxes'][i], caption)
        draw_box(image, annotations['bboxes'][i], color=c)
'''

'\ndef draw_annotations(image, annotations, color=(0, 255, 0), label_to_name=None):\n    """ Draws annotations in an image.\n\n    # Arguments\n        image         : The image to draw on.\n        annotations   : A [N, 5] matrix (x1, y1, x2, y2, label) or dictionary containing bboxes (shaped [N, 4]) and labels (shaped [N]).\n        color         : The color of the boxes. By default the color from keras_retinanet.utils.colors.label_color will be used.\n        label_to_name : (optional) Functor for mapping a label to a name.\n    """\n    if isinstance(annotations, np.ndarray):\n        annotations = {\'bboxes\': annotations[:, :4], \'labels\': annotations[:, 4]}\n\n    assert(\'bboxes\' in annotations)\n    assert(\'labels\' in annotations)\n    assert(annotations[\'bboxes\'].shape[0] == annotations[\'labels\'].shape[0])\n\n    for i in range(annotations[\'bboxes\'].shape[0]):\n        label   = annotations[\'labels\'][i]\n        c       = color if color is not None else label_colo

In [ ]:
'''
def _get_annotations(generator):
    """ Get the ground truth annotations from the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = annotations[num_detections, 5]

    # Arguments
        generator : The generator used to retrieve ground truth annotations.
    # Returns
        A list of lists containing the annotations for each image in the generator.
    """
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Parsing annotations: '):
        # load the annotations
        annotations = generator.load_annotations(i)

        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            if not generator.has_label(label):
                continue

            all_annotations[i][label] = annotations['bboxes'][annotations['labels'] == label, :].copy()

    return all_annotations
'''    

'\ndef _get_annotations(generator):\n    """ Get the ground truth annotations from the generator.\n\n    The result is a list of lists such that the size is:\n        all_detections[num_images][num_classes] = annotations[num_detections, 5]\n\n    # Arguments\n        generator : The generator used to retrieve ground truth annotations.\n    # Returns\n        A list of lists containing the annotations for each image in the generator.\n    """\n    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]\n\n    for i in progressbar.progressbar(range(generator.size()), prefix=\'Parsing annotations: \'):\n        # load the annotations\n        annotations = generator.load_annotations(i)\n\n        # copy detections to all_annotations\n        for label in range(generator.num_classes()):\n            if not generator.has_label(label):\n                continue\n\n            all_annotations[i][label] = annotations[\'bboxes\'][annotations[\'labels\

#Eval

In [ ]:
def evaluate(
    generator,
    model,
    iou_threshold=0.5,
    score_threshold=0.05,
    max_detections=100,
    save_path=None
):
    """ Evaluate a given dataset using a given model.

    # Arguments
        generator       : The generator that represents the dataset to evaluate.
        model           : The model to evaluate.
        iou_threshold   : The threshold used to consider when a detection is positive or negative.
        score_threshold : The score confidence threshold to use for detections.
        max_detections  : The maximum number of detections to use per image.
        save_path       : The path to save images with visualized detections to.
    # Returns
        A dict mapping class names to mAP scores.
    """
    # gather all detections and annotations
    all_detections, all_inferences = _get_detections(generator, model, score_threshold=score_threshold, max_detections=max_detections, save_path=save_path)
    all_annotations    = _get_annotations(generator)
    average_precisions = {}

    # all_detections = pickle.load(open('all_detections.pkl', 'rb'))
    # all_annotations = pickle.load(open('all_annotations.pkl', 'rb'))
    # pickle.dump(all_detections, open('all_detections.pkl', 'wb'))
    # pickle.dump(all_annotations, open('all_annotations.pkl', 'wb'))

    # process detections and annotations
    for label in range(generator.num_classes()-1):
        if not generator.has_label(label):
            continue

        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap         = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0, 0
            continue

        # sort by score
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision
        recall    = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision  = _compute_ap(recall, precision)
        average_precisions[label] = average_precision, num_annotations

    # inference time
    inference_time = np.sum(all_inferences) / generator.size()

    return average_precisions, inference_time


In [ ]:
def _get_annotations(generator):
    """ Get the ground truth annotations from the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = annotations[num_detections, 5]

    # Arguments
        generator : The generator used to retrieve ground truth annotations.
    # Returns
        A list of lists containing the annotations for each image in the generator.
    """
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Parsing annotations: '):
        # load the annotations
        annotations = generator.load_annotations(i)

        # copy detections to all_annotations
        for label in range(generator.num_classes()-1):
            if not generator.has_label(label):
                continue

            all_annotations[i][label] = annotations['bboxes'][annotations['labels'] == label, :].copy()

    return all_annotations

In [ ]:
def _get_detections(generator, model, score_threshold=0.05, max_detections=100, save_path=None):
    """ Get the detections from the model using the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = detections[num_detections, 4 + num_classes]

    # Arguments
        generator       : The generator used to run images through the model.
        model           : The model to run on the images.
        score_threshold : The score confidence threshold to use.
        max_detections  : The maximum number of detections to use per image.
        save_path       : The path to save the images with visualized detections to.
    # Returns
        A list of lists containing the detections for each image in the generator.
    """
    #save_path = None
    all_detections = [[None for i in range(generator.num_classes()) if generator.has_label(i)] for j in range(generator.size())]
    all_inferences = [None for i in range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Running network: '):
        print("i",i)
        raw_image    = generator.load_image(i)
        image, scale = generator.resize_image(raw_image.copy())
        image = generator.preprocess_image(image)

        if keras.backend.image_data_format() == 'channels_first':
            image = image.transpose((2, 0, 1))

        # run network
        start = time.time()
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))[:3]
        inference_time = time.time() - start

        # correct boxes for image scale
        boxes /= scale

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]

        # select those scores
        scores = scores[0][indices]

        # find the order with which to sort the scores
        scores_sort = np.argsort(-scores)[:max_detections]

        # select detections
        image_boxes      = boxes[0, indices[scores_sort], :]
        image_scores     = scores[scores_sort]
        image_labels     = labels[0, indices[scores_sort]]
        image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

        if save_path is not None:
            draw_annotations(raw_image, generator.load_annotations(i), label_to_name=generator.label_to_name)
            draw_detections(raw_image, image_boxes, image_scores, image_labels, label_to_name=generator.label_to_name, score_threshold=score_threshold)

            #cv2.imwrite(os.path.join(save_path, '{}.png'.format(i)), raw_image)

            #CHANGE HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
            #loop
            _annotations = generator.load_annotations(i)
            bounding_boxes = _annotations.get('bboxes')

            for b in bounding_boxes:
              true_box = b
              draw_box(draw, true_box, color=(255, 255, 0))

            THRES_SCORE = 0.5  
            draw__detections(draw, boxes, scores, labels, generator)
            cv2.imwrite(os.path.join(save_path, '{}.png'.format(i)), raw_image)
            
            ###################################################################


        # copy detections to all_detections
        for label in range(generator.num_classes()-1):
          if not generator.has_label(label):
            continue
          all_detections[i][label] = image_detections[image_detections[:, -1] == label, :-1]
          print("label",label)
        all_inferences[i] = inference_time

    return all_detections, all_inferences

In [ ]:
def _compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.

    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

#Evaluate

In [ ]:
def create_generator(image_min_side, image_max_side, dataset_type, annotations , classes, config, preprocess_image):
    """ Create generators for evaluation.
    """
    no_resize = False 
    group_method = 'ratio'
    common_args = {
        'config'           : config,
        'image_min_side'   : image_min_side,
        'image_max_side'   : image_max_side,
        'no_resize'        : no_resize,
        'preprocess_image' : preprocess_image,
        'group_method'     : group_method
    }
  

    if dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        validation_generator = CocoGenerator(
            args.coco_path,
            'val2017',
            shuffle_groups=False,
            **common_args
        )
    elif dataset_type == 'pascal':
        validation_generator = PascalVocGenerator(
            args.pascal_path,
            'test',
            image_extension=args.image_extension,
            shuffle_groups=False,
            **common_args
        )
    elif dataset_type == 'csv':
        validation_generator = CSVGenerator(
            annotations,
            classes,
            shuffle_groups=False,
            **common_args
        )
    else:
        raise ValueError('Invalid data type received: {}'.format(args.dataset_type))

    return validation_generator



In [ ]:
def parse_args(args):
    """ Parse the arguments.
    """
    parser     = argparse.ArgumentParser(description='Evaluation script for a RetinaNet network.')
    subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    subparsers.required = True

    coco_parser = subparsers.add_parser('coco')
    coco_parser.add_argument('coco_path', help='Path to dataset directory (ie. /tmp/COCO).')

    pascal_parser = subparsers.add_parser('pascal')
    pascal_parser.add_argument('pascal_path', help='Path to dataset directory (ie. /tmp/VOCdevkit).')
    pascal_parser.add_argument('--image-extension',   help='Declares the dataset images\' extension.', default='.jpg')

    csv_parser = subparsers.add_parser('csv')
    csv_parser.add_argument('annotations', help='Path to CSV file containing annotations for evaluation.')
    csv_parser.add_argument('classes', help='Path to a CSV file containing class label mapping.')

    parser.add_argument('model',              help='Path to RetinaNet model.')
    parser.add_argument('--convert-model',    help='Convert the model to an inference model (ie. the input is a training model).', action='store_true')
    parser.add_argument('--backbone',         help='The backbone of the model.', default='resnet50')
    parser.add_argument('--gpu',              help='Id of the GPU to use (as reported by nvidia-smi).')
    parser.add_argument('--score-threshold',  help='Threshold on score to filter detections with (defaults to 0.05).', default=0.05, type=float)
    parser.add_argument('--iou-threshold',    help='IoU Threshold to count for a positive detection (defaults to 0.5).', default=0.5, type=float)
    parser.add_argument('--max-detections',   help='Max Detections per image (defaults to 100).', default=100, type=int)
    parser.add_argument('--save-path',        help='Path for saving images with detections (doesn\'t work for COCO).')
    parser.add_argument('--image-min-side',   help='Rescale the image so the smallest side is min_side.', type=int, default=800)
    parser.add_argument('--image-max-side',   help='Rescale the image if the largest side is larger than max_side.', type=int, default=1333)
    parser.add_argument('--no-resize',        help='Don''t rescale the image.', action='store_true')
    parser.add_argument('--config',           help='Path to a configuration parameters .ini file (only used with --convert-model).')
    parser.add_argument('--group-method',     help='Determines how images are grouped together', type=str, default='ratio', choices=['none', 'random', 'ratio'])

    return parser.parse_args(args)

In [ ]:
def main(dataset_type, annotations, classes, model_path, save_path):
  #backbone_name = 'resnet50'
  image_min_side = 800
  image_max_side = 1333
  convert_model = True
  # parse arguments
  '''
  if args is None:
    args = sys.argv[1:]
    args = parse_args(args)
  '''

  # make sure tensorflow is the minimum required version
  check_tf_version()

  # optionally choose specific GPU
  '''
  if args.gpu:
    setup_gpu(args.gpu)
  '''
  # make save path if it doesn't exist

  if save_path is not None and not os.path.exists(save_path):
      os.makedirs(args.save_path)

  # optionally load config parameters
  config = '/content/Config/config.ini'
  
  if config:
    config = read_config_file(config)
  
  # create the generator
  backbone = models.backbone('resnet50')
  generator = create_generator(image_min_side, image_max_side, dataset_type, annotations , classes, config,backbone.preprocess_image)

  # optionally load anchor parameters
  anchor_params = None
  pyramid_levels = None

  if config and 'anchor_parameters' in config:
      anchor_params = parse_anchor_parameters(config)
  '''
  if args.config and 'pyramid_levels' in args.config:
      pyramid_levels = parse_pyramid_levels(args.config)
  '''
  # load the model
  print('Loading model, this may take a second...')
  model = models.load_model(model_path, backbone_name='resnet50')
  generator.compute_shapes = make_shapes_callback(model)

  # optionally convert the model
  if convert_model:
    model = models.convert_model(model, anchor_params=anchor_params, pyramid_levels=pyramid_levels)
    #model = models.convert_model(model)

  # print model summary
  #print(model.summary())
  print(generator.num_classes())
  # start evaluation
  
  #print(generator.load_annotations(19));
  

  if dataset_type == 'coco':
    from keras_retinanet.utils.coco_eval import evaluate_coco
    evaluate_coco(generator, model, args.score_threshold)
  else:
    average_precisions, inference_time = evaluate(
        generator,
        model,
        iou_threshold=0.5,
        score_threshold= 0.05,
        max_detections=100,
        save_path= '/content/saved_samples'
        )

  # print evaluation
  total_instances = []
  precisions = []
  for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
    generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)

  if sum(total_instances) == 0:
    print('No test instances found.')
    return

  print('Inference time for {:.0f} images: {:.4f}'.format(generator.size(), inference_time))

  print('mAP using the weighted average of precisions among classes: {:.4f}'.format(sum([a * b for a, b in zip(total_instances, precisions)]) / sum(total_instances)))
  print('mAP: {:.4f}'.format(sum(precisions) / sum(x > 0 for x in total_instances)))
  

# Predictions

In [ ]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [ ]:
THRES_SCORE = 0.5

def draw__detections(image, boxes, scores, labels, generator):

  #print(boxes[0], scores[0], labels[0])
  for box, score, label in zip(boxes[0], scores, labels[0]):
    
    if score < THRES_SCORE:
        break

    color = label_color(label)
    #print(box)
    #print(score)
    #print(label," ", labels_to_names[label-1])

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(generator.label_to_name(label), score)
    draw_caption(image, b, caption)


In [ ]:
def show_detected_objects(image_row):
  img_path = image_row.image_name
  
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  #loop
  true_box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]
  #print(true_box)
  draw_box(draw, true_box, color=(255, 255, 0))
  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

#Unzip blender dataset

In [ ]:
!unzip /content/sun90-20210510T124021Z-001.zip -d /content/sun90

unzip:  cannot find or open /content/sun90-20210510T124021Z-001.zip, /content/sun90-20210510T124021Z-001.zip.zip or /content/sun90-20210510T124021Z-001.zip.ZIP.


In [ ]:
ANNOTATIONS_FILE = '/content/sun90/sun90/sun90.csv'
CLASSES_FILE = '/content/sun90/sun90/labels.csv'

In [ ]:
%cd - 
%cd sun90/
%cd sun90/
!ls

/content
[Errno 2] No such file or directory: 'sun90/'
/content
[Errno 2] No such file or directory: 'sun90/'
/content
keras-retinanet  sample_data


#Download the EMT dataset from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/EMTD.zip -d /content/EMTD_ALL


#ANNOTATIONS_FILE = '/content/EMTD_ALL/EMTD/Detection/GT39_val_2.csv'
#CLASSES_FILE = '/content/EMTD_ALL/EMTD/Detection/labels.csv'

In [ ]:
!unzip /content/day-20210607T222247Z-001.zip -d /content/Day

In [ ]:
ANNOTATIONS_FILE = '/content/EMTD_ALL/EMTD/Detection/GT39_test420.csv'
CLASSES_FILE = '/content/EMTD_ALL/EMTD/Detection/labels.csv'

In [ ]:
ANNOTATIONS_FILE ='/content/Day/day/Daynoblur.csv'
CLASSES_FILE = '/content/Day/day/labels.csv'

In [ ]:
%cd -
%cd EMTD_ALL/
%cd EMTD/
%cd Detection/
%ls

In [ ]:
!ls

In [ ]:
main('csv',  ANNOTATIONS_FILE, CLASSES_FILE, '/content/drive/MyDrive/Resnet_Models/resnet50_csv_75.h5', '/content/saved_samples')

In [ ]:
!zip -r /content/EMTVD_samples.zip /content/saved_samples